In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

data = pd.read_csv('alcohol.csv')
data.head()

,Unnamed: 0,abuse,status,unemrate,age,educ,married,famsize,white,exhealth,...,mothalc,fathalc,livealc,inwf,employ,agesq,beertaxsq,cigtaxsq,ethanolsq,educsq
0,1,1,1,4.0,50,4,1,1,1,0,...,0,0,0,0,0,2500,0.111556,1444.0,4.159397,16
1,2,0,3,4.0,37,12,1,5,1,0,...,0,0,0,1,1,1369,0.111556,1444.0,4.159397,144
2,3,0,3,4.0,53,9,1,3,1,1,...,0,0,0,1,1,2809,0.111556,1444.0,4.159397,81
3,4,0,3,3.3,59,11,1,1,1,1,...,0,0,0,1,1,3481,0.057600,676.0,6.002402,121
4,5,0,3,3.3,43,10,1,1,1,1,...,0,1,1,1,1,1849,0.057600,676.0,6.002402,100


## EDA

In [2]:
import plotly.graph_objects as go

# Count combinations
counts = data.groupby(['abuse', 'employ']).size().reset_index(name='count')

# Define nodes
labels = ['Abuse=0', 'Abuse=1', 'Employ=0', 'Employ=1']

# Map source and target indices
source = []
target = []
value = []

for _, row in counts.iterrows():
    s = 0 if row['abuse'] == 0 else 1
    t = 2 if row['employ'] == 0 else 3
    source.append(s)
    target.append(t)
    value.append(row['count'])

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(label=labels),
    link=dict(source=source, target=target, value=value)
)])

fig.update_layout(title_text="Abuse vs Employment Sankey Plot", font_size=12)
fig.show(renderer='browser')


## Modelling

(i) What fraction of the sample is employed at the time of the interview? What fraction of the sample has abused alcohol?

In [3]:
employed_frac = data['employ'].mean()
abuse_frac = data['abuse'].mean()
employed_frac, abuse_frac

(np.float64(0.8981877418041132), np.float64(0.09916513948279373))

(ii) Run the simple regression of employ on abuse and report the results in the usual form, obtaining the heteroskedasticity-robust standard errors. Interpret the estimated equation. Is the relationship as you expected? Is it statistically significant?

In [4]:
import statsmodels.api as sm
X = sm.add_constant(data['abuse'])
y = data['employ']
model = sm.OLS(y, X).fit(cov_type='HC1')  # robust SE
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 employ   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.452
Date:                Thu, 23 Oct 2025   Prob (F-statistic):             0.0111
Time:                        20:20:55   Log-Likelihood:                -2185.9
No. Observations:                9822   AIC:                             4376.
Df Residuals:                    9820   BIC:                             4390.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9010      0.003    283.733      0.0

(iii) Run a probit of employ on abuse. Do you get the same sign and statistical significance as in part (i1)? How does the average partial effect for the probit compare with that for the linear probability model?

In [5]:
probit_model = sm.Probit(y, X).fit()
print(probit_model.summary(), '\n\n')

mfx = probit_model.get_margeff(method='dydx', at='overall')  # 'overall' = APE
print(mfx.summary())

Optimization terminated successfully.
         Current function value: 0.328678
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:                 employ   No. Observations:                 9822
Model:                         Probit   Df Residuals:                     9820
Method:                           MLE   Df Model:                            1
Date:                Thu, 23 Oct 2025   Pseudo R-squ.:                0.001120
Time:                        20:20:55   Log-Likelihood:                -3228.3
converged:                       True   LL-Null:                       -3231.9
Covariance Type:            nonrobust   LLR p-value:                  0.007140
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2872      0.018     70.630      0.000       1.252       1.323
abuse         -0.1480      0.

(iv) Obtain the fitted values for the LPM estimated in part ii) and report what they are when abuse = 0 and when abuse = 1. How do these compare to the probit fitted values, and why?

In [6]:
fitted_lpm_0 = model.predict([1, 0])[0]
fitted_lpm_1 = model.predict([1, 1])[0]
fitted_probit_0 = probit_model.predict([1, 0])[0]
fitted_probit_1 = probit_model.predict([1, 1])[0]

print(fitted_lpm_0, fitted_lpm_1)
print(fitted_probit_0, fitted_probit_1)

0.900994575045222 0.8726899383983704
0.900994575045208 0.8726899383983573


(v) To the LPM in part (i) add the variables age, agesq, educ, educsq, married, famsize, white, northeast, midwest, south, centcity, outercity, qrtl, qrt2, and qrt3. What happens to the coefficient on abuse and its statistical significance?

In [17]:
controls = ['age','agesq','educ','educsq','married','famsize','white','northeast',
            'midwest','south','centcity','outercity','qrt1','qrt2','qrt3', 'ethanol', "exhealth","vghealth","goodhealth","fairhealth"]
X2 = sm.add_constant(data[['abuse'] + controls])
model2 = sm.OLS(y, X2).fit(cov_type='HC1')
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 employ   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     43.43
Date:                Thu, 23 Oct 2025   Prob (F-statistic):          2.96e-171
Time:                        20:26:58   Log-Likelihood:                -1361.6
No. Observations:                9822   AIC:                             2767.
Df Residuals:                    9800   BIC:                             2925.
Df Model:                          21                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2214      0.078     -2.839      0.0

(vi) Estimate a probit model using the variables in part (v). Find the APE of abuse and its t statistic. Is the estimated effect now identical to that for the linear model? Is it "close"?

In [ ]:
probit2 = sm.Probit(y, X2).fit()
# Compute APE for abuse
pred_prob2 = probit2.predict(X2)
print(probit2.summary(), '\n\n')

mfx2 = probit2.get_margeff(method='dydx', at='overall')  # 'overall' = APE
print(mfx2.summary())

Optimization terminated successfully.
         Current function value: 0.298295
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:                 employ   No. Observations:                 9822
Model:                         Probit   Df Residuals:                     9805
Method:                           MLE   Df Model:                           16
Date:                Thu, 23 Oct 2025   Pseudo R-squ.:                 0.09346
Time:                        20:16:22   Log-Likelihood:                -2929.9
converged:                       True   LL-Null:                       -3231.9
Covariance Type:            nonrobust   LLR p-value:                3.110e-118
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7628      0.376     -4.689      0.000      -2.500      -1.026
abuse         -0.1208      0.

(vii) Variables indicating the overall health of each man are also included in the data set. Is it obvious that such variables should be included as controls? Explain.


(viii) Why might abuse be properly thought of as endogenous in the employ equation? Do you think the variables mothale and fathalc, indicating whether a man's mother or father were alcoholics, are sensible instrumental variables for abuse?

(ix) Estimate the LPM underlying part (v) by 2SLS, where mothale and fathalc act as IVs for abuse. Is the difference between the 2SLS and OLS coefficients practically large? 

In [18]:
from linearmodels.iv import IV2SLS
iv_model = IV2SLS(y, X2[['const'] + controls], data['abuse'], data[['mothalc','fathalc', 'livealc', 'beertax']]).fit(cov_type='robust')
print(iv_model)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 employ   R-squared:                      0.0751
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0731
No. Observations:                9822   F-statistic:                    883.48
Date:                Thu, Oct 23 2025   P-value (F-stat)                0.0000
Time:                        20:27:39   Distribution:                 chi2(21)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.2440     0.0809    -3.0152     0.0026     -0.4027     -0.0854
age            0.0177     0.0032     5.6003     0.00

(x) Use the test described in Section 15.5 to test whether abuse is endogenous in the LPM.

In [19]:
# Instruments and controls
instruments = ['mothalc', 'fathalc', 'beertax', 'livealc']

X_first = sm.add_constant(data[instruments + controls])
y_first = data['abuse']
first_stage = sm.OLS(y_first, X_first).fit()
data['resid_abuse'] = first_stage.resid

X_second = sm.add_constant(data[['abuse'] + controls + ['resid_abuse']])
y_second = data['employ']
second_stage = sm.OLS(y_second, X_second).fit(cov_type='HC1')
print(second_stage.summary())

                            OLS Regression Results                            
Dep. Variable:                 employ   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     41.74
Date:                Thu, 23 Oct 2025   Prob (F-statistic):          1.23e-171
Time:                        20:28:00   Log-Likelihood:                -1359.5
No. Observations:                9822   AIC:                             2765.
Df Residuals:                    9799   BIC:                             2930.
Df Model:                          22                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.2440      0.079     -3.090      